# ML - Problem Set 2: Predicting Poverty

El objetivo principal es construir un modelo predictivo de la pobreza de los hogares. Nótese que un hogar se clasifica como

\begin{equation}
    Poor = I \left( Inc < Pl \right)
\end{equation}

donde $I$ es una función indicadora que toma uno si el ingreso familiar está por debajo de una determinada línea de pobreza.

**texto en negrita**

In [5]:
# Importar librerias

import pandas as pd
from sklearn.impute import SimpleImputer

In [2]:
# Importar datos

sample = pd.read_csv('sample_submission.csv')
test_hogares = pd.read_csv('test_hogares.csv')
train_hogares = pd.read_csv('train_hogares.csv')
test_personas = pd.read_csv('test_personas.csv')
train_personas = pd.read_csv('train_personas.csv')

In [13]:
# Limpiar datos

common_columns_hogares = train_hogares.columns.intersection(test_hogares.columns) #obtener columnas comunes entre train_hogares y test_hogares
common_columns_personas = train_personas.columns.intersection(test_personas.columns) #obtener columnas comunes entre train_personas y test_personas
train_hogares = train_hogares[common_columns_hogares] #filtrar las columnas para alinear los datos
train_personas = train_personas[common_columns_personas] #filtrar las columnas para alinear los datos

columnas_deseadas = ['id', 'Orden', 'Clase', 'Dominio', 'P6020', 'P6040', 'P6050', 'P6090', 'P6100',
                     'P6210', 'P6210s1', 'P6240', 'Oficio', 'P6426', 'P6430', 'P6590', 'P6800',
                     'P6870', 'P6920', 'P7090', 'P7110', 'P7120', 'P7350', 'P7495', 'P7510s5',
                     'Pet', 'Oc', 'Des', 'Ina', 'Fex_c', 'Depto', 'Fex_dpto'] #datos mas relevantes

train_personas = train_personas[columnas_deseadas]
test_personas = test_personas[columnas_deseadas]

In [15]:
# Imputar datos falntantes

# Crear imputadores
imputador_moda = SimpleImputer(strategy='most_frequent')  # Para variables categóricas
imputador_mediana = SimpleImputer(strategy='median')  # Para variables numéricas

# Función para imputar datos
def imputar_datos(df):
    # Separar columnas numéricas y categóricas
    columnas_numericas = df.select_dtypes(include=['number']).columns
    columnas_categoricas = df.select_dtypes(exclude=['number']).columns

    # Imputar numéricas con la mediana
    if not columnas_numericas.empty:
        df[columnas_numericas] = imputador_mediana.fit_transform(df[columnas_numericas])

    # Imputar categóricas con la moda
    if not columnas_categoricas.empty:
        df[columnas_categoricas] = imputador_moda.fit_transform(df[columnas_categoricas])

# Aplicar imputación a todos los DataFrames
imputar_datos(test_hogares)
imputar_datos(train_hogares)
imputar_datos(test_personas)
imputar_datos(train_personas)

# Verificar si hay valores nulos restantes
print("Valores nulos restantes:")
print(f"test_hogares: {test_hogares.isnull().sum().sum()}")
print(f"train_hogares: {train_hogares.isnull().sum().sum()}")
print(f"test_personas: {test_personas.isnull().sum().sum()}")
print(f"train_personas: {train_personas.isnull().sum().sum()}")

Valores nulos restantes:
test_hogares: 0
train_hogares: 0
test_personas: 0
train_personas: 0
